In [2]:
# ok so today is 5 December
# I am reading
# https://www.aleksagordic.com/blog/vllm

# My goal for today. go over part1
# Fundamentals of vLLM:
# I want to learn how exactly
# 1) scheduling
# 2) paged attention
# 3) continious batching works

# less abstractions, mostly actual implementation code
# if I can trace fully one single inference
# that will be a huge success


In [4]:
%%capture 
!pip install vllm

In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          20G   20G  868K 100% /
tmpfs            64M     0   64M   0% /dev
/dev/md0         20G   13G  7.5G  63% /workspace
shm              94G  4.0K   94G   1% /dev/shm
/dev/nvme2n1p2  228G   58G  158G  27% /usr/bin/nvidia-smi
tmpfs           252G     0  252G   0% /sys/fs/cgroup
tmpfs           252G   12K  252G   1% /proc/driver/nvidia
tmpfs           252G  4.0K  252G   1% /etc/nvidia/nvidia-application-profiles-rc.d
tmpfs            51G   20M   51G   1% /run/nvidia-persistenced/socket
tmpfs           252G     0  252G   0% /proc/asound
tmpfs           252G     0  252G   0% /proc/acpi
tmpfs           252G     0  252G   0% /proc/scsi
tmpfs           252G     0  252G   0% /sys/firmware
tmpfs           252G     0  252G   0% /sys/devices/virtual/powercap


In [17]:
!nvidia-smi

Fri Dec  5 21:51:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0 Off |                  Off |
|  0%   25C    P8             23W /  450W |   22384MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
import os
import shutil

# Clear ALL caches aggressively
for path in [
    os.path.expanduser("~/.cache/torch"),
    os.path.expanduser("~/.cache/vllm"),
    os.path.expanduser("~/.cache/huggingface"),
    "/root/.cache/torch",
    "/root/.cache/vllm",
]:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed {path}")
        
import os
os.environ["VLLM_USE_V1"] = "0"
os.environ["VLLM_LOGGING_LEVEL"] = "ERROR"
os.environ["HF_HOME"] = "/workspace/hf_cache"
os.environ["TORCH_HOME"] = "/workspace/torch_cache"
os.environ["VLLM_CACHE_ROOT"] = "/workspace/vllm_cache"

from vllm import LLM
llm = LLM("deepseek-ai/deepseek-coder-6.7b-instruct",
          disable_log_stats=True,
          max_model_len=8192) # hard learned lesson 

[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


(EngineCore_DP0 pid=11558) [rank0]:W1205 18:12:48.257000 11558 torch/_inductor/codecache.py:1021] [0/0] fx graph cache unable to load compiled graph
(EngineCore_DP0 pid=11558) [rank0]:W1205 18:12:48.257000 11558 torch/_inductor/codecache.py:1021] [0/0] Traceback (most recent call last):
(EngineCore_DP0 pid=11558) [rank0]:W1205 18:12:48.257000 11558 torch/_inductor/codecache.py:1021] [0/0]   File "/usr/local/lib/python3.10/dist-packages/torch/_inductor/codecache.py", line 1019, in iterate_over_candidates
(EngineCore_DP0 pid=11558) [rank0]:W1205 18:12:48.257000 11558 torch/_inductor/codecache.py:1021] [0/0]     yield pickle.loads(content), content
(EngineCore_DP0 pid=11558) [rank0]:W1205 18:12:48.257000 11558 torch/_inductor/codecache.py:1021] [0/0] EOFError: Ran out of input
Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:02<00:00, 21.83it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 25.93it/s]


In [6]:
from vllm import SamplingParams

prompts = ["Explain PagedAttention in vLLM inference","Write a Python function to implement binary search."]
sampling_params = SamplingParams(temperature=0.7,max_tokens=256)

outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(f"Prompt: {output.prompt}")
    print(f"Output: {output.outputs[0].text}")
    print("-" * 40)

Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: Explain PagedAttention in vLLM inference
Output: .


PagedAttention refers to a method used in the LLM (Large Language Model) inference process. It's a technique used to manage the memory used by the model during inference. The idea is to divide the input into smaller chunks, process each chunk separately, and then stitch the results together. This way, the model doesn't need to load the entire input into memory at once, which can significantly reduce the memory footprint and speed up the inference process.

In the context of LLM inference, PagedAttention is used in the attention mechanism of the model. This mechanism is responsible for weighing the importance of different parts of the input during the inference process. By breaking the input into smaller chunks, the attention mechanism can compute the importance of different parts of the input without having to load the entire input into memory at once.

The PagedAttention method is an important component of the Hugging Face T

In [7]:
# hey looking at that output: does vLLM apply chat template at all?

In [8]:
llm.generate??

Signature:
llm.generate(
    prompts: str | vllm.inputs.data.TextPrompt | vllm.inputs.data.TokensPrompt | vllm.inputs.data.EmbedsPrompt | vllm.inputs.data.ExplicitEncoderDecoderPrompt | collections.abc.Sequence[str | vllm.inputs.data.TextPrompt | vllm.inputs.data.TokensPrompt | vllm.inputs.data.EmbedsPrompt | vllm.inputs.data.ExplicitEncoderDecoderPrompt],
    sampling_params: vllm.sampling_params.SamplingParams | collections.abc.Sequence[vllm.sampling_params.SamplingParams] | None = None,
    *,
    use_tqdm: bool | collections.abc.Callable[..., tqdm.auto.tqdm] = True,
    lora_request: list[vllm.lora.request.LoRARequest] | vllm.lora.request.LoRARequest | None = None,
    priority: list[int] | None = None,
) -> list[vllm.outputs.RequestOutput]
Source:   
    def generate(
        self,
        prompts: PromptType | Sequence[PromptType],
        sampling_params: SamplingParams | Sequence[SamplingParams] | None = None,
        *,
        use_tqdm: bool | Callable[..., tqdm] = True,
    

In [10]:
# ok like what is self ??
llm.__init__??

Signature:
llm.__init__(
    model: str,
    *,
    runner: Literal['auto', 'generate', 'pooling', 'draft'] = 'auto',
    convert: Literal['auto', 'none', 'embed', 'classify', 'reward'] = 'auto',
    tokenizer: str | None = None,
    tokenizer_mode: Union[Literal['auto', 'hf', 'slow', 'mistral'], str] = 'auto',
    skip_tokenizer_init: bool = False,
    trust_remote_code: bool = False,
    allowed_local_media_path: str = '',
    allowed_media_domains: list[str] | None = None,
    tensor_parallel_size: int = 1,
    dtype: Literal['auto', 'half', 'float16', 'bfloat16', 'float', 'float32'] = 'auto',
    quantization: Optional[Literal['awq', 'deepspeedfp', 'tpu_int8', 'fp8', 'ptpc_fp8', 'fbgemm_fp8', 'fp_quant', 'modelopt', 'modelopt_fp4', 'bitblas', 'gguf', 'gptq_marlin_24', 'gptq_marlin', 'gptq_bitblas', 'awq_marlin', 'gptq', 'compressed-tensors', 'bitsandbytes', 'hqq', 'experts_int8', 'ipex', 'quark', 'moe_wna16', 'torchao', 'auto-round', 'rtn', 'inc', 'mxfp4', 'petit_nvfp4', 'cpu_gptq'

In [ ]:
# ok so I went from most of this __init__ method and paused 
# at most crucial moment
# self.llm_engine = LLMEngine.from_engine_args(
#             engine_args=engine_args, usage_context=UsageContext.LLM_CLASS
# )

# it looks like this LLMEngine class decides everything

In [16]:
# ok now I want to seee how I got crushed by kvCache
from vllm.config.model import _get_and_verify_max_len
_get_and_verify_max_len??

Signature:
_get_and_verify_max_len(
    hf_config: Any,
    tokenizer_config: dict | None,
    max_model_len: int | None,
    disable_sliding_window: bool,
    sliding_window: int | None,
    spec_target_max_model_len: int | None = None,
    encoder_config: Optional[Any] = None,
) -> int
Source:   
def _get_and_verify_max_len(
    hf_config: PretrainedConfig,
    tokenizer_config: dict | None,
    max_model_len: int | None,
    disable_sliding_window: bool,
    sliding_window: int | None,
    spec_target_max_model_len: int | None = None,
    encoder_config: Any | None = None,
) -> int:
    """Get and verify the model's maximum length."""
    derived_max_model_len = float("inf")
    possible_keys = [
        # OPT
        "max_position_embeddings",
        # GPT-2
        "n_positions",
        # MPT
        "max_seq_len",
        # ChatGLM2
        "seq_length",
        # Command-R
        "model_max_length",
        # Whisper
        "max_target_positions",
        # Others
        "m

In [11]:
from vllm.v1.engine.llm_engine import LLMEngine
LLMEngine??

Init signature:
LLMEngine(
    vllm_config: vllm.config.vllm.VllmConfig,
    executor_class: type[vllm.v1.executor.abstract.Executor],
    log_stats: bool,
    aggregate_engine_logging: bool = False,
    usage_context: vllm.usage.usage_lib.UsageContext = <UsageContext.ENGINE_CONTEXT: 'ENGINE_CONTEXT'>,
    stat_loggers: list[type['AggregateStatLoggerBase'] | collections.abc.Callable[[vllm.config.vllm.VllmConfig, int], 'StatLoggerBase']] | None = None,
    mm_registry: vllm.multimodal.registry.MultiModalRegistry = <vllm.multimodal.registry.MultiModalRegistry object at 0x781588d6d930>,
    use_cached_outputs: bool = False,
    multiprocess_mode: bool = False,
) -> None
Source:        
class LLMEngine:
    """Legacy LLMEngine for backwards compatibility."""

    def __init__(
        self,
        vllm_config: VllmConfig,
        executor_class: type[Executor],
        log_stats: bool,
        aggregate_engine_logging: bool = False,
        usage_context: UsageContext = UsageContext.ENGIN

In [9]:
# ok so it goes to _validate_and_add_requests come on
llm._validate_and_add_requests??
# the interesting things like it's not returning ? but why ?



Signature:
llm._validate_and_add_requests(
    prompts: str | vllm.inputs.data.TextPrompt | vllm.inputs.data.TokensPrompt | vllm.inputs.data.EmbedsPrompt | vllm.inputs.data.ExplicitEncoderDecoderPrompt | collections.abc.Sequence[str | vllm.inputs.data.TextPrompt | vllm.inputs.data.TokensPrompt | vllm.inputs.data.EmbedsPrompt | vllm.inputs.data.ExplicitEncoderDecoderPrompt] | vllm.inputs.data.DataPrompt,
    params: vllm.sampling_params.SamplingParams | collections.abc.Sequence[vllm.sampling_params.SamplingParams] | vllm.pooling_params.PoolingParams | collections.abc.Sequence[vllm.pooling_params.PoolingParams],
    *,
    use_tqdm: bool | collections.abc.Callable[..., tqdm.auto.tqdm] = True,
    lora_request: collections.abc.Sequence[vllm.lora.request.LoRARequest] | vllm.lora.request.LoRARequest | None,
    priority: list[int] | None = None,
) -> None
Docstring: <no docstring>
Source:   
    def _validate_and_add_requests(
        self,
        prompts: PromptType | Sequence[PromptType]

In [20]:
# Cache config
print(llm.llm_engine.cache_config.num_gpu_blocks)
print(llm.llm_engine.cache_config.num_cpu_blocks)
print(llm.llm_engine.cache_config.block_size)

# Model config
print(llm.llm_engine.model_config.max_model_len)



1007
None
16
8192
